In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark


In [ ]:
# import os
# # Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# # For example:
# # spark_version = 'spark-3.4.0'
# spark_version = 'spark-3.4.0'
# os.environ['SPARK_VERSION']=spark_version

# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
# !tar xf $SPARK_VERSION-bin-hadoop3.tgz
# !pip install -q findspark

# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
dataFrame = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

# display the DataFrame
dataFrame.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.

dataFrame.createOrReplaceTempView('home_sales')
dataFrame.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
""").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    296050.24|
|      2012|    298233.42|
|      2017|    296576.69|
|      2014|    299073.89|
|      2013|    299999.39|
|      2011|     302141.9|
|      2015|    307908.86|
|      2010|    296800.75|
+----------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND  bathrooms = 3
    GROUP BY date_built
""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND  bathrooms = 3
    GROUP BY date_built
""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.7754991054534912 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.

spark.catalog.listTables()

[Table(name='home_sales', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [ ]:
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 1.5338108539581299 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
dataFrame.write.partitionBy("date_built").parquet("has_partitioned", mode ="overwrite")

In [ ]:
# 11. Read the parquet formatted data.
hsp_df = spark.read.parquet('has_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.

# Read the parquet formatted data
hsp_df = spark.read.parquet('has_partitioned')

# Create a temporary view for the parquet data
hsp_df.createOrReplaceTempView('partitioned_home_sales')


In [ ]:
start_time = time.time()

# Run the query on the parquet DataFrame
result_df = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM partitioned_home_sales
    WHERE price >= 350000
""")

# Show the result
result_df.show()

# Calculate and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.6757938861846924 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.

spark.sql('uncache table home_sales')

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False